Using imgaug to generate 40 augmented images as well as their corresponding polygon annotations from the "sample garbage space" image and its corresponding polygon annotations xml file

In [1]:
pip install imgaug imageio lxml

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\snepo\appdata\local\programs\python\python311\lib\site-packages\hyperopt-0.2.7-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\snepo\appdata\local\programs\python\python311\lib\site-packages\numpydoc-1.6.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\snepo\appdata\local\programs\python\python311\lib\site-packages\pathlib-1.0.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\snepo\appd

In [2]:
import imgaug.augmenters as iaa
import numpy as np
import cv2
import os
import xml.etree.ElementTree as ET
from imgaug.augmentables.polys import Polygon, PolygonsOnImage
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage

# Load the image
image_path = "sample garbage space.jpg"
image = cv2.imread(image_path)

# Parse the XML annotation
xml_path = "annotations.xml"
tree = ET.parse(xml_path)
root = tree.getroot()

# Extract polygons from XML
polygons = []
for polygon in root.findall('.//polygon'):
    points = polygon.get('points').split(';')
    points = [tuple(map(float, point.split(','))) for point in points]
    polygons.append(Polygon(points))

# Convert polygons to imgaug format
polys_oi = PolygonsOnImage(polygons, shape=image.shape)

# Define augmentation pipeline
seq = iaa.Sequential([
    iaa.Fliplr(0.5),  # horizontal flip
    iaa.Flipud(0.5),  # vertical flip
    iaa.Affine(rotate=(-45, 45)),  # rotate
    iaa.Multiply((0.8, 1.2)),  # change brightness
])

# Apply augmentations
for i in range(20):  # Generate 20 augmented images
    image_aug, polys_aug = seq(image=image, polygons=polys_oi)

    # Save augmented image
    aug_image_path = f"image/output3/augmented_{i}.jpg"
    cv2.imwrite(aug_image_path, image_aug)

    # Save augmented annotation
    aug_root = ET.Element("annotations")
    aug_meta = ET.SubElement(aug_root, "meta")
    aug_image_elem = ET.SubElement(aug_root, "image", id="0", name=f"augmented_{i}.jpg", width=str(image.shape[1]), height=str(image.shape[0]))
    
    for polygon in polys_aug.polygons:
        points_str = ';'.join([f"{point[0]},{point[1]}" for point in polygon.exterior])
        ET.SubElement(aug_image_elem, "polygon", label="garbage", source="manual", occluded="0", points=points_str, z_order="0")

    aug_tree = ET.ElementTree(aug_root)
    aug_xml_path = f"image/output3/augmented_{i}.xml"
    aug_tree.write(aug_xml_path)

print("Augmentation complete. Check the 'output3' folder for results.")


Augmentation complete. Check the 'output3' folder for results.
